# Import garden.db

### 3.1 all plants in db

In [2]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('gardening.db')
c = conn.cursor()

import readline

# Connect to the database
conn = sqlite3.connect('gardening.db')
c = conn.cursor()

# Set up readline for console input
readline.parse_and_bind("tab: complete")
readline.set_history_length(1000)

# Define a function to execute a SQL query
def execute_query(query):
    try:
        print("=========================================")
        # Execute the SQL query
        c.execute(query)

        # Get column names
        column_names = [description[0] for description in c.description] if c.description else []

        # Print column names
        for col_name in column_names:
            print(col_name, end="\t")
        if column_names:
            print()  # New line after column names


        results = c.fetchall()

        # Print the results
        if len(results) == 0:
            print("No results found.")
        else:
            for row in results:
                print(row)

        print("=========================================")
        
        # Commit the changes to the database
        conn.commit()

        # Print a message for successful INSERT, UPDATE, or DELETE statements
        if c.rowcount > 0:
            print("Query executed successfully.")

        # Print the last inserted row ID for INSERT statements
        if query.lower().startswith('insert'):
            print("Last inserted row ID:", c.lastrowid)

    except sqlite3.Error as e:
        # Print an error message if the query fails
        print("An error occurred:", e)

# Loop to prompt the user for input
query_lines = []
while True:
    try:
        # Check if it's the beginning of a new command
        if not query_lines:
            prompt = "Enter a SQL query (or 'exit' to quit): \n"
        else:
            prompt = ''

        line = input(prompt)
        
        # If the user just presses Enter without typing anything, consider it as execution time
        if not line.strip() and query_lines:
            query = ' '.join(query_lines)
            query_lines.clear()
            execute_query(query)
        else:
            query_lines.append(line)

    except KeyboardInterrupt:
        # Handle Ctrl+C to exit the loop cleanly
        print("\nExiting...")
        break

    # Check if the user wants to exit
    if line.lower() == 'exit':
        break

    # Handle up arrow to load last query (this remains unchanged, but might need a different approach)
    if line == '':
        query = readline.get_history_item(readline.get_current_history_length())

    # Add the query to the history (this remains unchanged)
    readline.add_history(line)



pname	ptype	pdiameter	pmaxheight	
('חרוב מצוי', 'עץ פרי', 4, 5)
('זית', 'עץ פרי', 4.5, 4)
('ברוש', 'עץ נוי', 2, 7)
('דולב', 'עץ נוי', 5, 7)
('אורן ירושלים', 'עץ נוי', 3, 6)
('חבושית', 'שיח', 0.7, 1)
('רוזמרין', 'שיח', 0.7, 1.5)
('אחירותם', 'שיח', 1, 2)
('ורד', 'שיח', 1, 1.2)
('גפנית', 'מטפס', 1, 6)
('לנטנה לילכית', 'שיח', 0.7, 0.8)
('צבעוני ההרים', 'פרח בצל', 0.1, 0.5)
('תורמוס', 'חד שנתי', 0.2, 0.7)
('ציפורני חתול', 'חד שנתי', 0.1, 0.3)
('aaa', 'bbb', 1, 2)
('my_bush1', 'שיח', 1, 2.22)
('my_bush2', 'שיח', 1, 2.22)


In [3]:
query = """
SELECT pname
FROM plant
WHERE ptype IN ('עץ פרי', 'עץ נוי');
"""
execute_query(query)

pname	
('חרוב מצוי',)
('זית',)
('ברוש',)
('דולב',)
('אורן ירושלים',)


In [4]:
# Close the database connection
conn.close()


In [5]:
%load_ext sql

In [6]:
%sql sqlite:///gardening.db

In [7]:
%%sql
SELECT pname
FROM plant
WHERE ptype IN ('עץ פרי', 'עץ נוי');

 * sqlite:///gardening.db
Done.


pname
חרוב מצוי
זית
ברוש
דולב
אורן ירושלים
